# The Agent Loop

This notebook demonstrates how to build an AI agent using LangChain 1.0's `create_agent` API with Qdrant for vector storage. We'll:

1. **Set up Qdrant** - Create an in-memory vector database
2. **Build a document processing pipeline** - Load, chunk, and index documents
3. **Create tools** - Define a search tool the agent can use
4. **Build the agent** - Use LangChain's create_agent with a system prompt
5. **Extract reasoning** - Parse the agent's message history to see its "thinking"

## Setup

Before running this notebook, make sure you have:
- An OpenAI API key set as the `OPENAI_API_KEY` environment variable
- The required packages installed: `uv pip install langchain langchain-openai langchain-qdrant langchain-community langchain-text-splitters qdrant-client python-dotenv`

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify API key is set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

## Setting up Qdrant

Qdrant is a vector database that stores embeddings and enables similarity search. We're using in-memory mode for simplicity, but Qdrant can also run as a persistent server or cloud service.

Key concepts:
- **Collection**: A named set of vectors (like a table in a traditional database)
- **Vector dimensions**: Must match your embedding model (1536 for text-embedding-3-small)
- **Distance metric**: Cosine similarity is standard for text embeddings

In [ ]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# Initialize embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create in-memory Qdrant client
qdrant_client = QdrantClient(":memory:")

# Collection name
COLLECTION_NAME = "study_materials"

# Create collection with proper dimensions (1536 for text-embedding-3-small)
qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

# Create LangChain vector store wrapper
vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME,
    embedding=embeddings
)

print(f"Created Qdrant collection: {COLLECTION_NAME}")

## Document Processing Pipeline

Before we can search documents, we need to:
1. **Load** - Read the document from disk
2. **Chunk** - Split into smaller pieces that fit in context windows
3. **Embed** - Convert text to vectors
4. **Index** - Store in the vector database

LangChain provides loaders and splitters that handle this pipeline. The `RecursiveCharacterTextSplitter` tries to split on natural boundaries (paragraphs, sentences) rather than cutting mid-word.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

def index_document(file_path: str, doc_name: str):
    """Load, chunk, and index a document."""
    loader = TextLoader(file_path)
    documents = loader.load()

    # Split into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        length_function=len
    )
    chunks = text_splitter.split_documents(documents)

    # Add metadata
    for chunk in chunks:
        chunk.metadata['source'] = doc_name

    # Index in Qdrant
    vector_store.add_documents(chunks)

    return len(chunks)

print("Document indexing function ready")

## Index Sample Documents

Let's index all `.txt` and `.md` files from the documents directory. In a real application, this might run in the background on server startup.

In [ ]:
# Find documents directory
documents_dir = Path("documents")
if not documents_dir.exists():
    documents_dir = Path("../documents")  # Try parent if running from api/

if documents_dir.exists():
    # Get all .txt and .md files
    story_files = sorted(
        list(documents_dir.glob("*.txt")) +
        list(documents_dir.glob("*.md"))
    )
    
    print(f"Found {len(story_files)} documents to index\n")
    
    total_chunks = 0
    for filepath in story_files:
        doc_name = filepath.stem.replace("-", " ").replace("_", " ").title()
        num_chunks = index_document(str(filepath), doc_name)
        total_chunks += num_chunks
        print(f"Indexed {num_chunks} chunks from {doc_name}")
    
    print(f"\nTotal: {total_chunks} chunks indexed")
else:
    print("Documents directory not found. Create a 'documents' folder with .txt or .md files.")

## Creating Tools

An agent needs tools to interact with the world. The `@tool` decorator turns a Python function into something the agent can call. The docstring is crucial - it gets passed to the LLM so it knows when and how to use the tool.

Our search tool:
- Takes a query string
- Searches the vector store for similar chunks
- Returns formatted results with source attribution

In [ ]:
from langchain.tools import tool

@tool
def search_materials(query: str) -> str:
    """
    Search the indexed study materials for information about a topic.
    Use this when you need to find specific information from the
    student's uploaded study materials.

    Args:
        query: The search term or question to look up
    """
    results = vector_store.similarity_search(query, k=3)

    if not results:
        return "No relevant information found in study materials."

    # Format results with source attribution
    formatted = []
    for doc in results:
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"[From {source}]:\n{doc.page_content}")

    return "\n\n".join(formatted)

# Test the tool directly
print("Testing search tool...")
print(search_materials.invoke("Irene Adler"))

## Building the Agent

Now we create the agent using LangChain 1.0's `create_agent` API. The key components:

1. **Model**: The LLM that powers reasoning (gpt-4o-mini)
2. **Tools**: Functions the agent can call
3. **System prompt**: Instructions that shape the agent's behavior

The system prompt is critical. It tells the agent:
- What role it plays (StudyBuddy tutoring assistant)
- When to use tools (questions about study materials)
- When NOT to use tools (general knowledge questions)
- How to format responses (cite sources)

In [ ]:
from langchain.agents import create_agent

# System prompt
SYSTEM_PROMPT = """You are StudyBuddy, an AI tutoring assistant helping students study Sherlock Holmes stories.

IMPORTANT: The student has uploaded study materials (Sherlock Holmes stories) that you MUST search
before answering questions about characters, plots, or events. Always use the search_materials tool
first for any question about:
- Characters (Holmes, Watson, Irene Adler, Moriarty, etc.)
- Story plots or mysteries
- Specific events or quotes
- Anything related to the study materials

Only answer from general knowledge for questions completely unrelated to the study materials
(like math questions or general facts).

When you search, cite which story the information comes from."""

# Create tools list
tools = [search_materials]

# Create the agent using LangChain 1.0 API
agent = create_agent(
    model="gpt-4o-mini",
    tools=tools,
    system_prompt=SYSTEM_PROMPT
)

print("Agent created successfully!")

## Invoking the Agent

When we invoke the agent, it returns a response containing the full message history. This history shows us exactly what the agent did:

1. **HumanMessage**: The user's question
2. **AIMessage with tool_calls**: The agent deciding to use a tool
3. **ToolMessage**: The result from executing the tool
4. **AIMessage with content**: The final answer

LangChain 1.0 uses native tool calling (structured function calls) rather than text-based ReAct reasoning.

In [ ]:
# Ask a question that should trigger search
response = agent.invoke({
    "messages": [{"role": "user", "content": "Who is Irene Adler?"}]
})

# Look at the raw response structure
print("Message history:")
print("-" * 50)
for msg in response["messages"]:
    msg_type = getattr(msg, 'type', 'unknown')
    print(f"\n[{msg_type.upper()}]")
    
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"  Tool call: {tc['name']}({tc['args']})")
    
    if msg.content:
        content_preview = msg.content[:200] + "..." if len(msg.content) > 200 else msg.content
        print(f"  Content: {content_preview}")

## Extracting Reasoning

For transparency, we want to show users what the agent did - which tools it called, what queries it made, and what results it found. This builds trust and helps debug issues.

We parse the message history to extract:
- **Actions**: Which tools were called and with what inputs
- **Observations**: What the tools returned
- **Final Answer**: The agent's response to the user

In [ ]:
def extract_reasoning(response):
    """Extract reasoning trace and final answer from agent response."""
    messages = response["messages"]
    reasoning_parts = []
    final_answer = ""

    for msg in messages:
        msg_type = getattr(msg, 'type', None)

        # Skip the user message
        if msg_type == 'human':
            continue

        # Check for tool calls (agent deciding to use a tool)
        if msg_type == 'ai' and hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tool_call in msg.tool_calls:
                reasoning_parts.append(f"Action: {tool_call['name']}")
                reasoning_parts.append(f"Input: {tool_call['args']}")

        # Check for tool responses
        elif msg_type == 'tool':
            tool_name = getattr(msg, 'name', 'search_materials')
            content = msg.content if msg.content else ''
            # Truncate long content
            display_content = content[:500] + '...' if len(content) > 500 else content
            reasoning_parts.append(f"Observation from {tool_name}:\n{display_content}")

        # The final AI message is the answer (has content but no tool calls)
        elif msg_type == 'ai' and msg.content:
            tool_calls = getattr(msg, 'tool_calls', [])
            if not tool_calls:
                final_answer = msg.content

    reasoning = "\n\n".join(reasoning_parts) if reasoning_parts else None
    return {"answer": final_answer, "reasoning": reasoning}

# Test with a question that requires search
response = agent.invoke({
    "messages": [{"role": "user", "content": "What happened in A Scandal in Bohemia?"}]
})

result = extract_reasoning(response)

print("=" * 50)
print("REASONING TRACE")
print("=" * 50)
print(result["reasoning"])
print("\n" + "=" * 50)
print("FINAL ANSWER")
print("=" * 50)
print(result["answer"])

## Testing Different Question Types

Let's see how the agent handles different types of questions:

1. **Questions about study materials** - Should trigger search
2. **General knowledge questions** - Should answer directly

In [ ]:
def ask_agent(question: str):
    """Helper function to ask the agent and display results."""
    print(f"\nQuestion: {question}")
    print("-" * 50)
    
    response = agent.invoke({
        "messages": [{"role": "user", "content": question}]
    })
    
    result = extract_reasoning(response)
    
    if result["reasoning"]:
        print("[Used tools]")
    else:
        print("[Answered directly - no tools used]")
    
    print(f"\nAnswer: {result['answer'][:500]}..." if len(result['answer']) > 500 else f"\nAnswer: {result['answer']}")
    return result

# Questions that should trigger search
print("\n" + "=" * 60)
print("QUESTIONS ABOUT STUDY MATERIALS (should use search)")
print("=" * 60)

ask_agent("Who is Irene Adler and why is she significant?")
ask_agent("What was the mystery in The Red-Headed League?")

In [ ]:
# Questions that should be answered directly
print("\n" + "=" * 60)
print("GENERAL KNOWLEDGE QUESTIONS (should NOT use search)")
print("=" * 60)

ask_agent("What is 2 + 2?")
ask_agent("What is the capital of France?")

## What's Next?

This notebook demonstrated the core concepts of building an AI agent:

1. **Vector storage** with Qdrant for semantic search
2. **Document processing** with LangChain loaders and splitters
3. **Tool creation** with the `@tool` decorator
4. **Agent creation** with `create_agent`
5. **Reasoning extraction** from the message history

The full StudyBuddy v3 application wraps this in a FastAPI server with:
- Background document indexing
- Status polling endpoint
- Chat interface with "Show reasoning" toggle

In the next chapter, we'll rebuild using LangGraph for complete control over the agent's state machine, adding reflection, confidence scoring, and observability.